In [1]:
import pickle

with open('params', 'rb') as f:
    params = pickle.load(f)
    x = params['x']
    index2word = params['index2word']
    word2index = params['word2index']

In [2]:
import sys
sys.path.append('../')
import soykeyword

In [3]:
from soykeyword.lasso import LassoKeywordExtractor

lassobased_extractor = LassoKeywordExtractor(costs=[500, 200, 100, 50, 10, 5, 1, 0.1],
                                             min_tf=20, 
                                             min_df=10)

lassobased_extractor.train(x, index2word)

In [4]:
documents = lassobased_extractor.get_document_index('아이오아이')

In [5]:
keywords = lassobased_extractor.extract_from_docs(documents, minimum_number_of_keywords=30)
keywords

4 keywords extracted from 0.100 cost
4 keywords extracted from 1.000 cost
5 keywords extracted from 5.000 cost
4 keywords extracted from 10.000 cost
9 keywords extracted from 50.000 cost
18 keywords extracted from 100.000 cost
28 keywords extracted from 200.000 cost
43 keywords extracted from 500.000 cost


[KeywordScore(word='아이오아이', frequency=270, coefficient=17.56799565592231),
 KeywordScore(word='선의', frequency=40, coefficient=1.480434740926323),
 KeywordScore(word='산들', frequency=90, coefficient=1.2241056167059199),
 KeywordScore(word='뮤직', frequency=195, coefficient=1.216090529641175),
 KeywordScore(word='엠카운트다운', frequency=221, coefficient=0.99836522546522033),
 KeywordScore(word='뉴스1스타', frequency=357, coefficient=0.92455558152612871),
 KeywordScore(word='세련', frequency=192, coefficient=0.76807656230783894),
 KeywordScore(word='일산동구', frequency=36, coefficient=0.69521289522745144),
 KeywordScore(word='챔피언', frequency=105, coefficient=0.63119336175921004),
 KeywordScore(word='다비치', frequency=84, coefficient=0.62076241729543247),
 KeywordScore(word='프로듀스101', frequency=96, coefficient=0.50373584675884231),
 KeywordScore(word='키미', frequency=297, coefficient=0.41187845254582717),
 KeywordScore(word='강렬', frequency=352, coefficient=0.38900385786541064),
 KeywordScore(word='걸그룹', frequ

In [6]:
keywords = lassobased_extractor.extract_from_word('아이오아이', minimum_number_of_keywords=30)
keywords

28 keywords extracted from 0.100 cost
63 keywords extracted from 1.000 cost


[KeywordScore(word='너무너무너무', frequency=86, coefficient=3.8132975168887526),
 KeywordScore(word='선의', frequency=40, coefficient=3.2604006440081812),
 KeywordScore(word='산들', frequency=90, coefficient=2.4466903853776172),
 KeywordScore(word='엠카운트다운', frequency=221, coefficient=1.7619021790070657),
 KeywordScore(word='챔피언', frequency=105, coefficient=1.4891637673705018),
 KeywordScore(word='사나', frequency=46, coefficient=1.4170590575648272),
 KeywordScore(word='드림', frequency=119, coefficient=1.3410780535641496),
 KeywordScore(word='뮤직', frequency=195, coefficient=1.1836575474109008),
 KeywordScore(word='먹고', frequency=216, coefficient=1.1657719313664119),
 KeywordScore(word='완전체', frequency=77, coefficient=1.1155591095135293),
 KeywordScore(word='일산', frequency=194, coefficient=0.96751205535516127),
 KeywordScore(word='세련', frequency=192, coefficient=0.95004811471662309),
 KeywordScore(word='같이', frequency=918, coefficient=0.84651284424017548),
 KeywordScore(word='컴백', frequency=536, coe

In [7]:
keywords = lassobased_extractor.extract_from_word('최순실', minimum_number_of_keywords=30)
keywords

78 keywords extracted from 0.100 cost


[KeywordScore(word='게이트', frequency=303, coefficient=4.2027703102187708),
 KeywordScore(word='정유라', frequency=329, coefficient=2.0362347756007022),
 KeywordScore(word='고영태', frequency=65, coefficient=1.7180591473658491),
 KeywordScore(word='편파기소', frequency=54, coefficient=1.2875641826181596),
 KeywordScore(word='최서원', frequency=94, coefficient=1.2771949514616516),
 KeywordScore(word='비선', frequency=288, coefficient=1.0127743879437674),
 KeywordScore(word='씨와', frequency=372, coefficient=0.783932427655881),
 KeywordScore(word='이화여대', frequency=651, coefficient=0.72315015211976097),
 KeywordScore(word='송민순', frequency=821, coefficient=0.69070498927737445),
 KeywordScore(word='뉴스', frequency=4682, coefficient=0.5794079056616942),
 KeywordScore(word='연설문', frequency=204, coefficient=0.49718150920923787),
 KeywordScore(word='침묵', frequency=223, coefficient=0.42632486915181383),
 KeywordScore(word='박근혜', frequency=1445, coefficient=0.40829849079709901),
 KeywordScore(word='실세', frequency=30